In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


dataset_dir = "../data/new_data"  
img_size = 224  
batch_size = 16
epochs = 15


In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Found 7481 images belonging to 34 classes.
Found 1847 images belonging to 34 classes.
Number of classes: 34


In [3]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

# Base model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze base layers
base_model.trainable = False

# Add custom classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['normalization[0][0]']      

In [6]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)


Epoch 1/15
468/468 [==============================] - 230s 492ms/step - loss: 3.5560 - accuracy: 0.0323 - val_loss: 3.5218 - val_accuracy: 0.0319
Epoch 2/15
468/468 [==============================] - 199s 425ms/step - loss: 3.5573 - accuracy: 0.0305 - val_loss: 3.5210 - val_accuracy: 0.0314
Epoch 3/15
468/468 [==============================] - 207s 443ms/step - loss: 3.5531 - accuracy: 0.0319 - val_loss: 3.5201 - val_accuracy: 0.0319
Epoch 4/15
468/468 [==============================] - 205s 439ms/step - loss: 3.5582 - accuracy: 0.0302 - val_loss: 3.5204 - val_accuracy: 0.0319
Epoch 5/15
468/468 [==============================] - 206s 439ms/step - loss: 3.5621 - accuracy: 0.0318 - val_loss: 3.5218 - val_accuracy: 0.0319
Epoch 6/15
468/468 [==============================] - 202s 433ms/step - loss: 3.5559 - accuracy: 0.0326 - val_loss: 3.5202 - val_accuracy: 0.0319
Epoch 7/15
468/468 [==============================] - 206s 440ms/step - loss: 3.5578 - accuracy: 0.0287 - val_loss: 3.5203 -

In [ ]:

base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False


model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)